In [20]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install seaborn

In [ ]:
!pip install matplotlib

In [ ]:
pip install numpy 


In [29]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import streamlit as st

In [31]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_passenger_ids = test["PassengerId"]

In [32]:
for dataset in [train, test]:
    dataset["Age"].fillna(dataset["Age"].median(), inplace=True)
    dataset["Fare"].fillna(dataset["Fare"].median(), inplace=True)
    dataset["Embarked"].fillna(dataset["Embarked"].mode()[0], inplace=True)

In [33]:
for dataset in [train, test]:
    dataset["FamilySize"] = dataset["SibSp"] + dataset["Parch"] + 1
    dataset["IsAlone"] = 0
    dataset.loc[dataset["FamilySize"] == 1, "IsAlone"] = 1

# Extract Title from Name
for dataset in [train, test]:
    dataset["Title"] = dataset["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)
    dataset["Title"] = dataset["Title"].replace(
        ["Lady", "Countess", "Capt", "Col", "Don", "Dr", "Major", 
         "Rev", "Sir", "Jonkheer", "Dona"], "Rare"
    )
    dataset["Title"] = dataset["Title"].replace("Mlle", "Miss")
    dataset["Title"] = dataset["Title"].replace("Ms", "Miss")
    dataset["Title"] = dataset["Title"].replace("Mme", "Mrs")

In [34]:
le = LabelEncoder()
for col in ["Sex", "Embarked", "Title"]:
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

In [35]:
drop_cols = ["PassengerId", "Name", "Ticket", "Cabin"]
train = train.drop(drop_cols, axis=1)
test = test.drop(drop_cols, axis=1)

In [36]:
X = train.drop("Survived", axis=1)
y = train["Survived"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [37]:
# Logistic Regression
lr = LogisticRegression(max_iter=1000)

# Random Forest (tuned)
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=6,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

# XGBoost (tuned)
xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.01,
    max_depth=4,
    random_state=42
)

In [39]:
ensemble = VotingClassifier(
    estimators=[("lr", lr), ("rf", rf), ("xgb", xgb)],
    voting="soft"
)

ensemble.fit(X_train, y_train)
val_pred = ensemble.predict(X_val)

print("Validation Accuracy:", accuracy_score(y_val, val_pred))


Validation Accuracy: 0.8212290502793296


In [27]:
final_pred = ensemble.predict(test)

output = pd.DataFrame({
    "PassengerId": test_passenger_ids,
    "Survived": final_pred
})
output.to_csv("submission.csv", index=False)
print("✅ Submission file created: submission.csv")

✅ Submission file created: submission.csv
